In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import json
import numpy as np
from cmath import exp
import pandas as pd
import cvxpy as cp
import csv
import time

In [ ]:
data = pd.read_csv("binance_data6.csv")
data
# Compute the covariance between each asset so that we can consider this values as part of our portfolio diversification constraint

# Unique asset list
asset_list = data["Asset"].unique()
# expected return
exp_ret = {}
return_list = []
for asset in asset_list:
    open_price = np.array(data[data["Asset"] == asset]["Open"].astype("float"))
    close_price = np.array(
        data[data["Asset"] == asset]["Close"].astype("float"))

    # Sign will be used to indicate the value gradient direction
    returns = ((close_price - open_price)/open_price)
    exp_ret[asset] = returns.mean()
    return_list.append(returns)

# Expected return on each asset
mu = [i for i in exp_ret.values()]

# Compute covariance between returns
sigma = np.cov(np.vstack(return_list))
# Here mu is the value associated with the expected average return for each asset
for i, v in zip(asset_list, mu):
    print(f"Expected average return for asset {i} is {v}")
# It is important to know what the cost is of each asset so that we can also limit the budget we would like to spend in our investment.
filter = data.groupby("Asset").agg({"Open time": max}).reset_index()
costs = data.merge(filter, how='inner').drop_duplicates()
# print(costs)
cost_list = costs[["Asset", "Open"]].to_dict('records')
print(cost_list)

In [ ]:
# Serializing json
data = {"mu": mu, "sigma": sigma.tolist(), "assets": cost_list}
json_object = json.dumps(data, indent=4)

with open("binance-data6.json", "w") as file:
    file.write(json_object)

data = None
with open("binance-data6.json", "r") as jsonfile:
    data = json.load(jsonfile)

returns = data['mu']
covar = data['sigma']

assets = []
costs = []
for row in data['assets']:
    assets.append(row["Asset"])
    costs.append(float(row["Open"]))

# Half the money
budget = np.sum(costs)/0.5
sns.barplot(y=returns, x=assets)
plt.show()
# We will use CVX and its Python implementation (cvxpy) with a mixed integer optimization
# approach as our problem is restricted by the boolean values our $x$ variables can take,
# created for convex optimization; itself isn't a solver, it is a modeling framework

In [ ]:
x_val = cp.Variable(len(returns), boolean=True)
theta = cp.Parameter(nonneg=True)
ret = np.array(returns)@x_val
risk = cp.quad_form(x_val, covar)
e_costs = np.array(costs)@x_val

# Constraints
cons = [cp.sum(x_val) >= 0, cp.sum(e_costs) <= budget, x_val >= 0]

# Objective function
obj = cp.Minimize(- ret + theta*risk)

# Problem
prob = cp.Problem(obj, cons)
#time
i = 0
with open('cvxpy6.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    field = ['number of asset',"iteration", "time"]
    writer.writerow(field)
    while i<1000:
        i = i + 1
        start = time.process_timey()
        theta.value = 0.03 # This is related to the risk penalty lagrangian
        prob.solve(solver='ECOS_BB')
        end = time.process_time()
        writer.writerow([6, i, end-start])
data1 = pd.read_csv('cvxpy6.csv')
print(data1['time'].mean())
print(data1['time'].std())
for i, val in enumerate(np.round(x_val.value,1)):
    if val == 1:
        print(f"Asset {assets[i]} was selected")
    else:
        print(f"Asset {assets[i]} was not selected")